Given a sequence of {F|L|R}, each of which is "move forward one step", "turn left, then move forward one step", "turn right, then move forward one step":
1. which tours are closed?
2. what is the area enclosed by the tour?

In [2]:
import collections
import enum
import random
import os

import matplotlib.pyplot as plt
%matplotlib inline


In [3]:
class Direction(enum.Enum):
    UP = 1
    RIGHT = 2
    DOWN = 3
    LEFT = 4
    
turn_lefts = {Direction.UP: Direction.LEFT, Direction.LEFT: Direction.DOWN,
              Direction.DOWN: Direction.RIGHT, Direction.RIGHT: Direction.UP}

turn_rights = {Direction.UP: Direction.RIGHT, Direction.RIGHT: Direction.DOWN,
               Direction.DOWN: Direction.LEFT, Direction.LEFT: Direction.UP}

def turn_left(d):
    return turn_lefts[d]

def turn_right(d):
    return turn_rights[d]


In [4]:
Step = collections.namedtuple('Step', ['x', 'y', 'dir'])
Mistake = collections.namedtuple('Mistake', ['i', 'step'])

In [5]:
def advance(step, d):
    if d == Direction.UP:
        return Step(step.x, step.y+1, d)
    elif d == Direction.DOWN:
        return Step(step.x, step.y-1, d)
    elif d == Direction.LEFT:
        return Step(step.x-1, step.y, d)
    elif d == Direction.RIGHT:
        return Step(step.x+1, step.y, d)

In [6]:
def step(s, current):
    if s == 'F':
        return advance(current, current.dir)
    elif s == 'L':
        return advance(current, turn_left(current.dir))
    elif s == 'R':
        return advance(current, turn_right(current.dir))
    else:
        raise ValueError

In [7]:
def trace_tour(tour, startx=0, starty=0, startdir=Direction.RIGHT):
    current = Step(startx, starty, startdir)
    trace = [current]
    for s in tour:
        current = step(s, current)
        trace += [current]
    return trace    

In [8]:
def positions(trace):
    return [(s.x, s.y) for s in trace]

In [9]:
def valid(trace):
    return (trace[-1].x == 0 
            and trace[-1].y == 0 
            and len(set(positions(trace))) + 1 == len(trace))

In [10]:
def valid_prefix(tour):
    current = Step(0, 0, Direction.RIGHT)
    prefix = []
    posns = []
    for s in tour:
        current = step(s, current)
        prefix += [s]
        if (current.x, current.y) in posns:
            return ''
        elif current.x == 0 and current.y == 0: 
            return ''.join(prefix)
        posns += [(current.x, current.y)]
    if current.x == 0 and current.y == 0:
        return ''.join(prefix)
    else:
        return ''

In [11]:
def mistake_positions(trace, debug=False):
    mistakes = []
    current = trace[0]
    posns = [(0, 0)]
    for i, current in enumerate(trace[1:]):
        if (current.x, current.y) in posns:
            if debug: print(i, current)
            mistakes += [Mistake(i+1, current)]
        posns += [(current.x, current.y)]
    if (current.x, current.y) == (0, 0):
        return mistakes[:-1]
    else:
        return mistakes + [Mistake(len(trace)+1, current)]

In [12]:
def returns_to_origin(mistake_positions):
    return [i for i, m in mistake_positions
           if (m.x, m.y) == (0, 0)]

In [13]:
sample_tours = ['FFLRLLFLRL', 'FLLFFLFFFLFFLFLLRRFR', 'FFRLLFRLLFFFRFLLRLLRRLLRLL']

In [14]:
def bounds(trace):
    return (max(s.x for s in trace),
            max(s.y for s in trace),
            min(s.x for s in trace),
            min(s.y for s in trace))

In [15]:
plot_wh = {Direction.UP: (0, 1), Direction.LEFT: (-1, 0),
           Direction.DOWN: (0, -1), Direction.RIGHT: (1, 0)}

In [16]:
def chunks(items, n=2):
    return [items[i:i+n] for i in range(len(items) - n + 1)]

In [17]:
def plot_trace(trace, colour='k', xybounds=None, fig=None, subplot_details=None, filename=None):
    plt.axis('on')
    plt.axes().set_aspect('equal')
    for s, t in chunks(trace, 2):
        w, h = plot_wh[t.dir]
        plt.arrow(s.x, s.y, w, h, head_width=0.1, head_length=0.1, fc=colour, ec=colour, length_includes_head=True)
    xh, yh, xl, yl = bounds(trace)
    if xybounds is not None:    
        bxh, byh, bxl, byl = xybounds
        plt.xlim([min(xl, bxl)-1, max(xh, bxh)+1])
        plt.ylim([min(yl, byl)-1, max(yh, byh)+1])
    else:
        plt.xlim([xl-1, xh+1])
        plt.ylim([yl-1, yh+1])
    if filename:
        plt.savefig(filename)

# Part 1

In [18]:
with open('06-tours.txt') as f:
    tours = [t.strip() for t in f.readlines()]
len(tours)

226

In [19]:
sum(len(t) for t in tours if valid(trace_tour(t)))

61762

In [31]:
sum(1 for t in tours if valid(trace_tour(t)))

100

In [20]:
sum(len(t) for t in tours)

123845

In [21]:
%%timeit
sum(len(t) for t in tours if valid(trace_tour(t)))

1 loop, best of 3: 211 ms per loop


# Part 2

In [23]:
# %%timeit
# [(i, j) 
#  for i, pi in enumerate(tours) 
#  for j, pj in enumerate(tours)
#  if i != j
#  if not valid(trace_tour(pi))
#  if not valid(trace_tour(pj))
#  if valid(trace_tour(pi + pj))]

In [ ]:
# [(i, j) 
#  for i, pi in enumerate(tours) 
#  for j, pj in enumerate(tours)
#  if i != j
#  if not valid(trace_tour(pi))
#  if not valid(trace_tour(pj))
#  if valid(trace_tour(pi + pj))]

In [ ]:
# (sum(len(t) for t in tours if valid(trace_tour(t)))
#     +
#     sum(len(pi + pj) 
#      for i, pi in enumerate(tours) 
#      for j, pj in enumerate(tours)
#      if i != j
#      if not valid(trace_tour(pi))
#      if not valid(trace_tour(pj))
#      if valid(trace_tour(pi + pj)))
# )

In [24]:
l1s = {}
for t in tours:
    tr = trace_tour(t)
    l1 = abs(tr[-1].x) + abs(tr[-1].y)
    if l1 > 0:
        if l1 not in l1s:
            l1s[l1] = []
        l1s[l1] += [t]

for l1 in l1s:
    if l1 < 20:
        print(l1, len(l1s[l1]))

1 1
2 1
3 4
4 5
5 7
6 3
7 1
8 2
9 2
11 2
18 1
19 1


In [25]:
[(l1, len(l1s[l1])) for l1 in l1s if l1 < 20]

[(1, 1),
 (2, 1),
 (3, 4),
 (4, 5),
 (5, 7),
 (6, 3),
 (7, 1),
 (8, 2),
 (9, 2),
 (11, 2),
 (18, 1),
 (19, 1)]

In [27]:
# %%timeit
# (sum(len(t) for t in tours if valid(trace_tour(t)))
#     +
#     sum(len(pi + pj) 
#      for i, pi in enumerate(tours) 
#      for j, pj in enumerate(tours)
#      if i != j
#      if not valid(trace_tour(pi))
#      if not valid(trace_tour(pj))
#      if valid(trace_tour(pi + pj)))
# )

In [28]:
good_is = []
goods = []
tried = []
for l1 in l1s:
    possible_l1s = [i for i in range(l1-1, l1+1) if i in l1s]
    candidates = [t for i in possible_l1s for t in l1s[i]]
    for t1 in candidates:
        for t2 in candidates:
            if t1 != t2:
                t12 = t1 + t2
                if (t12) not in tried:
                    tried += [(t12)]
                    if valid(trace_tour(t12)):
                        good_is += [(tours.index(t1), tours.index(t2))]
                        goods += [t12]

In [29]:
(sum(len(t) for t in tours if valid(trace_tour(t)))
    +
    sum(len(t12) for t12 in goods)
)

80622

In [32]:
%%timeit

l1s = {}
for t in tours:
    tr = trace_tour(t)
    l1 = abs(tr[-1].x) + abs(tr[-1].y)
    if l1 > 0:
        if l1 not in l1s:
            l1s[l1] = []
        l1s[l1] += [t]

goods = []
tried = []
for l1 in l1s:
    possible_l1s = [i for i in range(l1-1, l1+1) if i in l1s]
    candidates = [t for i in possible_l1s for t in l1s[i]]
    for t1 in candidates:
        for t2 in candidates:
            if t1 != t2:
                t12 = t1 + t2
                if (t12) not in tried:
                    tried += [(t12)]
                    if valid(trace_tour(t12)):
                        goods += [t12]

(sum(len(t) for t in tours if valid(trace_tour(t)))
    +
    sum(len(t12) for t12 in goods)
)

1 loop, best of 3: 1.27 s per loop


In [33]:
len(goods)

13

In [37]:
sorted(good_is)

[(16, 125),
 (70, 48),
 (91, 128),
 (110, 134),
 (116, 194),
 (123, 51),
 (136, 9),
 (142, 193),
 (152, 63),
 (168, 150),
 (201, 83),
 (208, 204),
 (212, 113)]

In [38]:
sorted(good_is, key=lambda p: p[1])

[(136, 9),
 (70, 48),
 (123, 51),
 (152, 63),
 (201, 83),
 (212, 113),
 (16, 125),
 (91, 128),
 (110, 134),
 (168, 150),
 (142, 193),
 (116, 194),
 (208, 204)]